## Use pretrained network to output keypoint's description

In [5]:
from __future__ import division, print_function
import glob
import os
import cv2
import PIL
import random
import numpy as np
# import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.nn.init
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
from tqdm import tqdm
from torch.autograd import Variable
from copy import deepcopy, copy
from config_profile import args
from Utils import cv2_scale36, cv2_scale, np_reshape, np_reshape64
from scipy.optimize import linear_sum_assignment
import sys

In [6]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
from descriptor_CNN3 import DesNet
model = DesNet()
model.eval()
if args.cuda:
    model.cuda()
weight_path = "checkpoint.pth"
trained_weight = torch.load(weight_path)
model.load_state_dict(trained_weight['state_dict'])


In [7]:
# load patches
# query
patches_p_dir = "../keypoint_detector/p_patches.pt"
patches_p = torch.load(patches_p_dir)
print("query patches: ", patches_p.shape)
patches_p =  patches_p.view(-1, 1, 32, 32).cuda()

# dataset
patches_q_dir = "../keypoint_detector/q_patches.pt"
patches_q = torch.load(patches_q_dir)
print("dataset patches: ", patches_q.shape)
patches_q =  patches_q.view(-1, 1, 32, 32).cuda()

query patches:  torch.Size([5, 100, 1, 32, 32])
dataset patches:  torch.Size([5, 100, 1, 32, 32])


In [8]:
# query
description_p = model(patches_p)
description_p = description_p.view(-1, 100, 128)

# dataset
description_q = model(patches_q)
description_q = description_q.view(-1, 100, 128)

In [9]:
print("p description: ", description_p.shape)
print("q description: ", description_q.shape) 

p description:  torch.Size([5, 100, 128])
q description:  torch.Size([5, 100, 128])


In [10]:
torch.save(description_p, "p_description.pt")
torch.save(description_q, "q_description.pt") 

In [11]:
c = torch.cat((description_p, description_q), 0)
torch.save(c, "feature_descriptors.pt")
print("feature_descriptors: ", c.shape)

feature_descriptors:  torch.Size([10, 100, 128])
